# Grid2x2

In [1]:
import csv
import os

import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor
import supersuit as ss

from evaluate import evaluate
from helper_functions import make_parallel_env

In [2]:
from stable_baselines3.common.utils import set_random_seed

TEST_NUM = 1
NET = "grid2x2" 
SEED = 23423    # default SUMO seed no.
STACK_NUM = 3   # frames to stack

set_random_seed(SEED)

## Create Environment and Instantiate Agent

In [3]:
from reward_functions import combined_reward
from observation import Grid2x2ObservationFunction

csv_dir = os.path.join("outputs", NET, f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

env_params = {
    "net_file": os.path.join("nets", NET, f"{NET}.net.xml"),
    "route_file": os.path.join("nets", NET, f"{NET}.rou.xml"),
    "num_seconds": 3600,
    "reward_fn": combined_reward,
    "sumo_seed": SEED,
    "observation_class": Grid2x2ObservationFunction
}
congestion_reward = combined_reward.__defaults__[0].__name__
alpha = combined_reward.__defaults__[1]  # congestion component coefficient
print(congestion_reward, alpha)

env = make_parallel_env(**env_params)
env.reset(seed=SEED)

env = ss.frame_stack_v1(env, STACK_NUM)
env = ss.pettingzoo_env_to_vec_env_v1(env)

# Create env for evaluation
eval_env = ss.concat_vec_envs_v1(env, num_vec_envs=1, base_class="stable_baselines3")
eval_env = VecMonitor(eval_env)

# Create env for training
train_env = ss.concat_vec_envs_v1(env, num_vec_envs=8, base_class="stable_baselines3")
train_env = VecMonitor(train_env)

delta_wait_time_reward 0.875


In [4]:
from hyperparams import custom

model = PPO(
    "MlpPolicy",
    train_env,
    tensorboard_log=os.path.join("logs", NET),
    verbose=1,
    **custom,
)

Using cuda device


## Evaluate Random Agent

In [5]:
# Evaluate untrained random agent

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", NET, "eval_untrained")

reward_untrained, _ = evaluate(model, eval_env, csv_path, tb_log_dir, n_eval_episodes=1)

print(reward_untrained)

-274654.4434541464


In [7]:
# Record results
untrained_csv = pd.read_csv(os.path.join(csv_dir, "untrained.csv"))
arrived_untrained = sum(untrained_csv.arrived_num)
pm_untrained = sum(untrained_csv.tyre_pm)
final_wait_untrained = untrained_csv.waiting_time.iat[-1]

data = [TEST_NUM, congestion_reward, alpha, 0,
        reward_untrained, arrived_untrained, pm_untrained, final_wait_untrained]

with open(f"{NET}_results.csv", "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

## Train Agent

In [8]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement

# Separate env for eval callback
callback_env = ss.concat_vec_envs_v1(env, num_vec_envs=1, base_class="stable_baselines3")
callback_env = VecMonitor(callback_env)

stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)
eval_callback = EvalCallback(callback_env, callback_after_eval=stop_train_callback, eval_freq=25000,
                             best_model_save_path=os.path.join("saved_models", NET, f"test_{TEST_NUM}"))

TRAIN_STEPS = 2e6
model.learn(total_timesteps=TRAIN_STEPS, callback=eval_callback)

Logging to logs\grid2x2\PPO_1
-----------------------------
| time/              |      |
|    fps             | 60   |
|    iterations      | 1    |
|    time_elapsed    | 136  |
|    total_timesteps | 8192 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 49           |
|    iterations           | 2            |
|    time_elapsed         | 328          |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0001770469 |
|    clip_fraction        | 0            |
|    clip_range           | 0.1          |
|    entropy_loss         | -1.39        |
|    explained_variance   | -9.18e-06    |
|    learning_rate        | 0.00025      |
|    loss                 | 4.14e+05     |
|    n_updates            | 4            |
|    policy_gradient_loss | -0.000646    |
|    value_loss           | 8.71e+05     |
-----------------------------

In [9]:
# Evaluate trained agent
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", NET, f"PPO_{TEST_NUM}", "eval_trained")

reward_trained, _ = evaluate(model, eval_env, csv_path, tb_log_dir, n_eval_episodes=1)

print(reward_trained)

-84072.53790447116


In [10]:
# Record results
trained_csv = pd.read_csv(os.path.join(csv_dir, "trained.csv"))
arrived_trained = sum(trained_csv.arrived_num)
pm_trained = sum(trained_csv.tyre_pm)
final_wait_trained = trained_csv.waiting_time.iat[-1]

data = [TEST_NUM, congestion_reward, alpha, TRAIN_STEPS,
        reward_trained, arrived_trained, pm_trained, final_wait_trained]

with open(f"{NET}_results.csv", "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [11]:
# Move untrained.csv into numbered directory
import shutil

original = os.path.join("logs", NET, "eval_untrained")
target = os.path.join("logs", NET, f"PPO_{TEST_NUM}","eval_untrained")

shutil.move(original, target)

'logs\\grid2x2\\PPO_1\\eval_untrained'

### Save Model

In [12]:
model.save(os.path.join("saved_models",f"PPO_{NET}_{TEST_NUM}"))

## Render Video

In [16]:
env_params.update({"use_gui": True})

render_env = make_parallel_env(**env_params)
render_env.reset(seed=SEED)
render_env = ss.frame_stack_v1(render_env, STACK_NUM)
render_env = ss.pettingzoo_env_to_vec_env_v1(render_env)
render_env = VecMonitor(render_env)

In [17]:
import pyautogui

obs = render_env.reset()

folder_path = os.path.join("renders", NET, f"test_{TEST_NUM}")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

max_time = render_env.unwrapped.par_env.unwrapped.env.sim_max_time
delta_time = render_env.unwrapped.par_env.unwrapped.env.delta_time
vid_length = int(max_time/delta_time)

for i in range(vid_length):
    actions, _ = model.predict(obs, deterministic=True)
    obs, rewards, dones, infos = render_env.step(actions)

    im = pyautogui.screenshot(
        os.path.join(folder_path,f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

In [18]:
# Clean up
render_env.close()

In [19]:
import subprocess

subprocess.run([
    "ffmpeg", "-y", "-r", "30", "-i",
    f"renders\\{NET}\\test_{TEST_NUM}\\img%d.jpg",
    f"videos\\{NET}_{TEST_NUM}.mp4"
])

CompletedProcess(args=['ffmpeg', '-y', '-r', '30', '-i', 'renders\\grid2x2\\test_1\\img%d.jpg', 'videos\\grid2x2_1.mp4'], returncode=0)